## 에어비엔비 가격?
- 평가: R-Squared, MAE, MSE, RMSE, RMSLE, MAPE
- target : price(가격)
- csv파일 생성 : 수험번호.csv (예시 아래 참조)
~~~
id,price
34323697,238
29927138,183
120362,234
~~~



- 1번째 풀이

In [202]:
# 1. 데이터 불러오기
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(train.shape, test.shape)

# print(train.head(5))
#          id                                           name    host_id  \
# 0  34323697  Sonder | The Biltmore | Bright 1BR + Sofa Bed  219517861

#       host_name neighbourhood_group     neighbourhood  latitude  longitude  \
# 0  Sonder (NYC)           Manhattan  Theater District  40.75965  -73.98652

#          room_type  minimum_nights  number_of_reviews last_review  \
# 0  Entire home/apt              29                  0         NaN

#    reviews_per_month  calculated_host_listings_count  availability_365
# 0                NaN                             327               338

# 2. EDA
# shape
# print(train.shape) # (39116, 16)
# print(test.shape) # (9779, 15)

# info()
# print(train.info())
# print(test.info())

# isnull().sum()
# print(train.isnull().sum())  # name, host_name / last_review, reviews_per_month
# print(test.isnull().sum())

# describe()
# print(train.describe())
# print(test.describe())
# print(train.describe(include='O'))
# print(test.describe(include='O'))

cols = train.select_dtypes(include='O').columns

# set(train[col]) set(test[col])
# print(set(train[cols]) == set(test[cols])) # True

# 3. 전처리
# last_review 타입 datetime으로 -> 모르겠음
# train['last_review'] = pd.to_datetime(train['last_review'])
# print(train.info())

# null 값 0으로
train = train.fillna(0)
test = test.fillna(0)

# id, name, host_name 컬럼 빼기 / test id는 test_id에 담아놓기
train = train.drop(['id', 'name', 'host_id', 'host_name'], axis=1)
test = test.drop(['name', 'host_id', 'host_name'], axis=1)

test_id = test.pop('id')

# 4. Feature Engineering
# int float : min-max, standard / Object : 단순 삭제, label, one-hot

# baseline 단순삭제
# cols = train.select_dtypes(include='O').columns
# print(cols)

# train = train.drop(cols, axis=1)
# test = test.drop(cols, axis=1)

# print(train.shape) # (39116, 9)
# print(test.shape) # (9779, 8)

# label
from sklearn.preprocessing import LabelEncoder

cols = ['neighbourhood_group', 'neighbourhood', 'room_type']

encoder = LabelEncoder()

for col in cols:
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])

cols = train.select_dtypes(include='O').columns
train = train.drop(cols, axis=1)
test = test.drop(cols, axis=1)

# print(train.info())

# 5. train-validation split
from sklearn.model_selection import train_test_split

y = train.pop('price')
X_tr, X_val, y_tr, y_val = train_test_split(train, y, test_size=0.1, random_state=123)

# print(y_tr.head(5))

# 6. 모델 선택, 학습, 예측
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=123)
model.fit(X_tr, y_tr)
pred = model.predict(X_val)

# print(pred)

# 7. 평가
from sklearn.metrics import mean_squared_error
import numpy as np

def rmse(x, y):
    return mean_squared_error(x, y)

# print(rmse(y_val, pred))

# baseline
# 35910.479610523886
# 35374.83741528914

# 8. test.csv 예측 결과 제출

pred = model.predict(test)


submit = pd.DataFrame({
    'id': test_id,
    'price': pred
})

submit.to_csv('0000.csv', index=False)

(39116, 16) (9779, 15)


In [203]:
y_test = pd.read_csv('y_test.csv')

rmse(y_test, pred)

40033.40272613083

## 데이터 불러오기

In [204]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(train.shape, test.shape)

(39116, 16) (9779, 15)


##EDA

In [205]:
train.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price
0,35742633,Luxury new 1 bed apartment in East Williamsburg,265866685,Mindy,Brooklyn,Bushwick,40.69796,-73.92915,Entire home/apt,30,0,NaN,NaN,2,179,198
1,15840089,Kid- (and Adult-) Friendly Uptown 2-Bedroom,99602138,Yolanda,Manhattan,Washington Heights,40.83490,-73.94829,Entire home/apt,3,36,2019-01-01,1.13,1,5,150


In [206]:
train.isnull().sum()

id                                   0
name                                14
host_id                              0
host_name                           16
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                    0
last_review                       8053
reviews_per_month                 8053
calculated_host_listings_count       0
availability_365                     0
price                                0
dtype: int64

In [207]:
test.isnull().sum()

id                                   0
name                                 2
host_id                              0
host_name                            5
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                    0
last_review                       1999
reviews_per_month                 1999
calculated_host_listings_count       0
availability_365                     0
dtype: int64

In [208]:
train['price'].describe()

count    39116.000000
mean       153.057547
std        248.758522
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max      10000.000000
Name: price, dtype: float64

In [209]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39116 entries, 0 to 39115
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              39116 non-null  int64  
 1   name                            39102 non-null  object 
 2   host_id                         39116 non-null  int64  
 3   host_name                       39100 non-null  object 
 4   neighbourhood_group             39116 non-null  object 
 5   neighbourhood                   39116 non-null  object 
 6   latitude                        39116 non-null  float64
 7   longitude                       39116 non-null  float64
 8   room_type                       39116 non-null  object 
 9   minimum_nights                  39116 non-null  int64  
 10  number_of_reviews               39116 non-null  int64  
 11  last_review                     31063 non-null  object 
 12  reviews_per_month               

## 데이터 전처리 & 피처엔지니어링

In [210]:
train.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price
0,35742633,Luxury new 1 bed apartment in East Williamsburg,265866685,Mindy,Brooklyn,Bushwick,40.69796,-73.92915,Entire home/apt,30,0,NaN,NaN,2,179,198
1,15840089,Kid- (and Adult-) Friendly Uptown 2-Bedroom,99602138,Yolanda,Manhattan,Washington Heights,40.83490,-73.94829,Entire home/apt,3,36,2019-01-01,1.13,1,5,150
2,24495605,Brooklyn Art Residence,75173477,Lara,Brooklyn,Bushwick,40.69932,-73.91191,Entire home/apt,3,7,2019-05-05,0.48,1,6,200


In [211]:
train.nunique()

# name, host_name, last_review, host_id 삭제
# reviews_per_month 0 으로 채우기

id                                39116
name                              38447
host_id                           30821
host_name                          9943
neighbourhood_group                   5
neighbourhood                       221
latitude                          17378
longitude                         13607
room_type                             3
minimum_nights                       94
number_of_reviews                   377
last_review                        1720
reviews_per_month                   903
calculated_host_listings_count       47
availability_365                    366
price                               638
dtype: int64

In [212]:
cols = ['name', 'host_id', 'host_name', 'last_review']
cols

['name', 'host_id', 'host_name', 'last_review']

In [213]:
train.shape

(39116, 16)

In [214]:
train = train.drop(cols, axis=1)
test = test.drop(cols, axis=1)

In [215]:
train.shape

(39116, 12)

In [216]:
train['reviews_per_month'] = train['reviews_per_month'].fillna(0)
test['reviews_per_month'] = test['reviews_per_month'].fillna(0)

In [217]:
train.isnull().sum()

id                                0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
price                             0
dtype: int64

In [218]:
train = train.drop('id', axis=1)
test_id = test.pop('id')

test.head(2)

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Manhattan,Theater District,40.75965,-73.98652,Entire home/apt,29,0,0.00,327,338
1,Brooklyn,Bedford-Stuyvesant,40.68062,-73.94418,Entire home/apt,30,1,0.75,6,284


In [219]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39116 entries, 0 to 39115
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood_group             39116 non-null  object 
 1   neighbourhood                   39116 non-null  object 
 2   latitude                        39116 non-null  float64
 3   longitude                       39116 non-null  float64
 4   room_type                       39116 non-null  object 
 5   minimum_nights                  39116 non-null  int64  
 6   number_of_reviews               39116 non-null  int64  
 7   reviews_per_month               39116 non-null  float64
 8   calculated_host_listings_count  39116 non-null  int64  
 9   availability_365                39116 non-null  int64  
 10  price                           39116 non-null  int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 3.3+ MB


In [220]:
from sklearn.preprocessing import LabelEncoder

cols = train.select_dtypes(include='O').columns
print(cols)

for col in cols:
    encoder = LabelEncoder()
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])

train[cols].head(2)

Index(['neighbourhood_group', 'neighbourhood', 'room_type'], dtype='object')


,neighbourhood_group,neighbourhood,room_type
0,1,28,0
1,2,206,0


## 검증 데이터 분리

In [221]:
from sklearn.model_selection import train_test_split

y = train.pop('price')

X_tr, X_val, y_tr, y_val = train_test_split(train, y, test_size=0.15, random_state=123)

In [222]:
X_tr.head()

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
20207,3,4,40.76129,-73.92009,0,1,56,2.73,1,126
8455,3,117,40.75174,-73.92967,0,7,0,0.00,1,0
23997,2,95,40.76269,-73.98928,0,2,4,4.00,1,29
7329,1,28,40.69725,-73.91181,2,1,0,0.00,1,0
32888,2,64,40.72771,-73.98983,1,2,66,2.39,3,27


## 모델 & 평가

### 회귀 평가지표
- MAE(Mean Absolute Error) : 실제값과 예측값 차이를 절대값으로 변환해 평균한 것
- MSE(Mean Squared Error) : 실제값과 예측값 차이를 제곱해 평균한 것
- RMSE(Root Mean Squared Error) : MSE에 루트를 씌운 것


- R2 Score : 회귀 모델의 설명력 -> 1에 가까울수록 좋음
- RMSLE(Root Mean Squared Log Error) : 예측값이 실제값보다 작을때 더 큰 패널티 부여
- MAPE(Mean Absolute Percentage Error) : MAE를 퍼센트로 표시

In [230]:
# 평가
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
def rmse(y_test, y_pred): #RMSE
    return np.sqrt(mean_squared_error(y_test, y_pred))

def rmsle(y_test, y_pred): #RMSLE
    return np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(y_pred), 2)))

def mape(y_test, y_pred): #MAPE
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [225]:
# 선형회귀
from sklearn.linear_model import LinearRegression

modle = LinearRegression()

model.fit(X_tr, y_tr)
pred = model.predict(X_val)

In [226]:
print("r2 : ",  r2_score(y_val, pred))
print("mae : ", mean_absolute_error(y_val, pred))
print("mse : ", mean_squared_error(y_val, pred))
print("rmes : ", rmse(y_val, pred))
print("rmsle : ", rmsle(y_val, pred))
print("mape : ", mape(y_val, pred))

r2 :  0.1656474921215063
mae :  67.74460293115202
mse :  53720.80398110468
rmes :  231.7774880809279
rmsle :  0.48614731886064916
mape :  inf


In [227]:
# 릿지
from sklearn.linear_model import Ridge

model = Ridge()
model.fit(X_tr,y_tr)
pred=model.predict(X_val)
print(r2_score(y_val, pred))
print("r2: ",r2_score(y_val, pred))
print("mae: ",mean_absolute_error(y_val, pred))
print("mse: ",mean_squared_error(y_val, pred))
print("rmse: ",rmse(y_val, pred))
print("rmsle: ",rmsle(y_val, pred))
print("mape: ",mape(y_val, pred))

0.07352078612738255
r2:  0.07352078612738255
mae:  75.1225745423621
mse:  59652.494324697334
rmse:  244.23860121753344
rmsle:  0.6228927733225624
mape:  inf


<ipython-input-223-c0f7fdd27b5a>:8: RuntimeWarning: invalid value encountered in log1p
  return np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(y_pred), 2)))


In [228]:
# 라쏘
from sklearn.linear_model import Lasso

model = Lasso()
model.fit(X_tr,y_tr)
pred=model.predict(X_val)
print("r2: ",r2_score(y_val, pred))
print("mae: ",mean_absolute_error(y_val, pred))
print("mse: ",mean_squared_error(y_val, pred))
print("rmse: ",rmse(y_val, pred))
print("rmsle: ",rmsle(y_val, pred))
print("mape: ",mape(y_val, pred))

r2:  0.06592496286246119
mae:  76.16357041264048
mse:  60141.56067115982
rmse:  245.2377635503142
rmsle:  0.6285426687241119
mape:  inf


<ipython-input-223-c0f7fdd27b5a>:8: RuntimeWarning: invalid value encountered in log1p
  return np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(y_pred), 2)))


In [231]:
#랜덤포레스트
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_tr, y_tr)
pred = model.predict(X_val)

print("r2: ",r2_score(y_val, pred))
print("mae: ",mean_absolute_error(y_val, pred))
print("mse: ",mean_squared_error(y_val, pred))
print("rmse: ",rmse(y_val, pred))
print("rmsle: ",rmsle(y_val, pred))
print("mape: ",mape(y_val, pred))

r2:  0.17990782925422266
mae:  67.52136730288571
mse:  52802.63477974514
rmse:  229.78823899352452
rmsle:  0.4854552947703311
mape:  inf


In [238]:
# xgb
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_tr, y_tr)
pred = model.predict(X_val)

print("r2: ",r2_score(y_val, pred))
print("mae: ",mean_absolute_error(y_val, pred))
print("mse: ",mean_squared_error(y_val, pred))
print("rmse: ",rmse(y_val, pred))
print("rmsle: ",rmsle(y_val, pred))
print("mape: ",mape(y_val, pred))

r2:  0.37768923791645037
mae:  67.57271598947844
mse:  40068.23752496083
rmse:  200.1705211187722
rmsle:  0.5232476335072022
mape:  inf


<ipython-input-230-c0f7fdd27b5a>:8: RuntimeWarning: invalid value encountered in log1p
  return np.sqrt(np.mean(np.power(np.log1p(y_test) - np.log1p(y_pred), 2)))


## 예측 및 csv 제출

In [235]:
test.head()

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,2,191,40.75965,-73.98652,0,29,0,0.00,327,338
1,1,13,40.68062,-73.94418,0,30,1,0.75,6,284
2,1,214,40.70665,-73.94061,0,4,50,0.51,1,331
3,1,13,40.68427,-73.95460,1,1,18,3.33,1,16
4,2,94,40.82028,-73.95381,1,1,3,0.08,1,0


In [239]:
pred = model.predict(test)
pred

array([358.99228, 119.63793, 149.56647, ..., 135.99306, 188.206  ,
       191.28525], dtype=float32)

In [241]:
submit = pd.DataFrame({
    'id': test_id,
    'price': pred
})

submit.to_csv('0000.csv', index=False)

##정리

In [243]:
y_test = pd.read_csv("y_test.csv")
print(r2_score(y_test, pred))

-0.00989070902187561
